# Installing twint
Installing twint will install all related packages (like numpy,etc) that makes twint function properly.

### Upgrading twint
For those who already have twint and wish to upgrade it due to certain functionality not working, or any other reasons, run the uninstall command first.

Otherwise, for a fresh install, you may skip running the code below and start from the second cell

In [1]:

!pip3 uninstall twint --yes


Found existing installation: twint 2.1.20
Uninstalling twint-2.1.20:
  Successfully uninstalled twint-2.1.20


In [2]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master #egg=twint

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-req-build-_lb73rkj
  Running command git clone -q https://github.com/twintproject/twint.git /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-req-build-_lb73rkj
  Running command git checkout -q origin/master
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Created wheel for twint: filename=twint-2.1.21-py3-none-any.whl size=38760 sha256=e07ff87bfe6cad5c2b948cca583b2cd4bd3a38fc2bdd9d8b37661cb02d9eca27
  Stored in directory: /private/var/folders/1j/w13pmgjd6r59876tb75jxls80000gn/T/pip-ephem-wheel-cache-zjln_dgb/wheels/8d/dc/9f/74b4483d5f997036f04aec7f42bd4b3c80f04264920c368068
Successfully built twint
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' comma

In [2]:
!pip3 install nest_asyncio

You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [13]:
!pip3 install python-dotenv

  Using cached python_dotenv-0.15.0-py2.py3-none-any.whl (18 kB)
You should consider upgrading via the '/Users/pcowe/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
!pip freeze > requirements.txt

# Import Packages
Importing modules and essential packages for running this notebook :)

In [15]:
import os, sys, time, csv, math
# Basic utilities for processing data in python
import pandas as pd
import numpy as np
# Open source Twitter intelligence tool
import twint
# Twitter authentication client
from twitterclient import get_twitter_client
# Twitter API client
from tweepy import Cursor
# Removes runtimeError in jupyterlab -> This event loop is already running
import nest_asyncio
nest_asyncio.apply()

# Find network actors using twint
Use the cell below to gather information/data about different search queries, hasthags, etc to identify which actors and conversations you wish to study.

All data from this section will be stored in '/search queries' folder

In [2]:
%%capture
# Configure twint query parameters to find all users talking about FSL

qp = twint.Config()
qp.Search = "French as a second language"
qp.Output = "data/search queries/fsl-terms.csv"
qp.Store_csv = True

# Run twint on configured qp

twint.run.Search(qp)
print("Running search query")

NameError: name 'twint' is not defined

# Extract username list of network actors
In this step we need to identify all actors that we will include in the network we are going to study. To do this, we will extract usernames from all the files created in the /search queries folder 

In [23]:
# variables
MAX_FRIENDS = 15000
client = get_twitter_client()
max_pages = math.ceil(MAX_FRIENDS/ 5000)

In [24]:
#setup paginate function
def paginate(items, n):
    """
    Generate n-sized chunks from Items

    """
    for i in range(0, len(items), n):
        yield items[i:i+n]

In [25]:
#Open csv file to extract column names
with open('data/search queries/fsl-terms.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    columnNames = []
    for row in csv_reader:
        columnNames.append(row)
        #break after reading first row
        break
columnNames = columnNames[0]


In [34]:
# Todo: Add code to extract list of username from multiple csv files

data = pd.read_csv('data/search queries/fsl-terms.csv', names=columnNames)
usernames = data.username.tolist()


'anyechka'

In [40]:
print(usernames[2])

enbikawa


# Creating Edges

In the next part I will create directed edges from our network of users. This is directed because we are goimg to represent followers/following relationships for each user in our general group of users.


In [ ]:
# make sure you are not repeating already scraped users. You will waste time
# for this, check your network list for users you already have scraped and remove them from usernames 

ignore_these_users = pd.read_csv('data/network/network-edges', names=['From', 'To'])


In [41]:
#loop through all users and write their followers to csv

# len(explored) + len(skipped) = len(usernames) 
explored_users = []
skipped_users = []
try:
    
    for user in usernames:

        fname = "data/network/network-edges.csv"
        fieldnames = ['From', 'To']
        with open(fname, 'a') as f:

            csv_writer = csv.DictWriter(f, fieldnames=fieldnames)

            # get a user's followers list
            for followers in Cursor(client.followers_ids, screen_name=user).pages(max_pages):
                for chunk in paginate(followers, 100):
                    userFollowers = client.lookup_users(user_ids=chunk)

                    for follower in userFollowers:

                        #print(len(users))

                        #print(follower.screen_name)
                        csv_writer.writerow({'From': follower.screen_name, 'To': user})

                    # f.write(json.dumps(user._json)+"\n")
                if len(followers) == 5000:
                    print("More results available. Sleeping for 60 seconds to avoid rate limit")
                    time.sleep(60)

            # get a user's following list
            for friends in Cursor(client.friends_ids, screen_name=user).pages(max_pages):

                for chunk in paginate(friends, 100):
                    userFollowing = client.lookup_users(user_ids=chunk)
                    for followed in userFollowing:
                        csv_writer.writerow({'From': user, 'To': followed.screen_name})
                if len(friends) == 5000:
                    print("More results available. Sleeping for 60 seconds to avoid rate limit")
                    time.sleep(60)

            f.close()
            
except tweepy.TweepError, e:
    if e == 
    print("Skipped {user}")
    pass


NameError: name 'TweepError' is not defined

# Creating Nodes

In the next part I will create nodes from our network actors. 

### Optional
Add code to include specific node attributes to enhance network visualization in gephi.


In [ ]:
# Add code for representing nodes in nodes.csv